In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

/home/marian/anaconda3/envs/ml/lib/python3.8/site-packages/fastbook/__init__.py:18: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
from fastbook import *